# 趣旨

個人的に再帰関数が好きなので、反復処理を書くさいには再帰関数にしている。そうすると「だいたいこういう風に書く」ということが予め決まってくるので、そういった話をメモしておく。

# 初歩の再帰関数

例えば「あるリストからnという要素以外を集めたリスト」を返す関数を考えるとしよう。このとき単純な再帰関数で実装すると次のようになる。

In [9]:
(define (simple-filter x lst)
  (cond
   [(null? lst) '()]
   [(not (equal? (car lst) x)) (cons (car lst) (simple-filter x (cdr lst)))]
   [else (simple-filter x (cdr lst))]))

(require rackunit)
(check-equal? (simple-filter 0 (list 1 3 0 4 5)) (list 1 3 4 5))

自分の場合、**再帰部分** と **入り口** は分けて書くことのほうが多い。これはOCamlで再帰を書くときに良く使われるパターンなんだけど、このようにして分けると色々とメリットが多いことがわかったので、普段からそのように書くことのほうが多い。

In [10]:
(define (simple-filter x lst)
  (define (inner lst)
  (cond
   [(null? lst) '()]
   [(not (equal? (car lst) x)) (cons (car lst) (inner (cdr lst)))]
   [else (inner (cdr lst))]))
  (inner lst))

(require rackunit)
(check-equal? (simple-filter 0 (list 1 3 0 4 5)) (list 1 3 4 5))

関数型言語の場合、何の変数が **更新される** のか、そして何の変数が **更新されないのか** を意識することが多いし、最近のプログラミング言語でも「更新してもよい変数」と「更新してはいけない変数」というものを明示的にするような宣言が導入されるようになっている。

局地的に関数を宣言することのメリットは、まず再帰にとって余計な引数を消すことが出来る（例えば上の例で言うならば **x** は関数全体の処理を通じて触れないため、局地関数では触れないようにしている）というのが挙げられる。

外に再帰関数を持つと大変で、こういった変わらない引数をいちいち渡さなくてはいけなくなる。

# 局地的グローバル変数

例えば、何らかの関数の処理で、ある処理の結果を集めるといったような場合、単純な再帰関数だと上手くいかないことが多い。もちろん、それらの結果をリストにしておいて、あとで`fold-left`で圧縮するというのも一つの手段ではあるのだけど、出来るだけオーダーを減らしたいよね、というときには「関数同士の間で共有される」ような変数を上で定義するのがいい。ちゃんとした用語はたぶんあるんだけど、個人的にはこれを **局地的グローバル変数** とか呼んでいる。

なぜ **局地的グローバル変数** かというと、あくまでトップレベルの変数は汚さず、その関数の内部だけで変数のスコープは完結しているが、その関数にとってはあたかも **グローバル変数** として振る舞うからだ、という感じである。

たとえば先の関数を無理やり「局地的グローバル変数」化すると次のようになる。

In [15]:
(define (simple-filter x lst)
  (let ([result (list)])
  (define (inner lst)
    (if (null? lst) result
     (begin 
      (when (not (equal? (car lst) x))
        (set! result (append result (list (car lst)))))
    (inner (cdr lst)))))
    (inner lst)))

(require rackunit)
(check-equal? (simple-filter 0 (list 1 3 0 4 5)) (list 1 3 4 5))

`simple-filter`の処理は直線的ではあるので、あまり **局地的グローバル変数** のありがたみを受けないし、これはいわゆる「末尾再帰バージョン」のにすれば、この **局地的グローバル変数** は削除してもよい（ちなみに **末尾再帰** に関しては余りにも多く語られているので、このドキュメントからは削除する）。

しかし、例えば直線的ではなく、いってしまえば　**「木構造」的な処理**　の場合、どうしても「全体の処理」にアクセスできないという状況がうまれる。なので、全体の処理と照らし合わせる場合はこのような **局地的グローバル変数** を通じて、関数同士の結果を照合するというのがいいだろう。ちゃんとした実例に関しては **継続手習い** という別のJupyter notebookに書いてあるので、それを参考されたし。